# Transfer Learning ด้วย Pretrained CONVNET

**Pretrained network** คือ convolutional network สำหรับการจำแนกประเภทของรูปภาพ ที่ผ่านการเทรนด้วยชุดข้อมูลขนาดใหญ่แล้ว โดยหากชุดข้อมูลที่ใช้เทรนมีขนาดใหญ่และมีความหลากหลายเพียงพอแล้ว ฟีเจอร์เชิงพื้นที่แบบมีลำดับขั้น (spatial hierarchy of features) ที่ pretrained convnet เรียนรู้ได้นั้น สามารถนำไปใช้เป็นโมเดลพื้นฐานสำหรับรูปทรงในการงานด้านการประมวลผลภาพ (computer vision) ปัญหาอื่น ที่ต่างไปจากเดิมได้ 

แนวความคิดในการนำ pretrained convnet ไปใช้ใน computer vision task ที่ต่างจาก computer vision task ที่โมเดลถูกสร้างขึ้นมานี้ เรียกว่า **transfer learning**

![การนำ Pretrained CONVNET มาใช้](images/fig1.jpg)

<center>**รูปที่ 1 การใช้ Pretrained convolutional neural network เป็น Feature Extractor สำหรับ classifier ตัวใหม่**</center>

จากรูปที่ 1 จะเห็นว่า เราจะไม่นำส่วนที่เป็น dense layers ในตอนท้ายของ convnet มาใช้ เนื่องจากเหตุผลหลักสองประการคือ ข้อแรกฟีเจอร์ที่เรียนรู้ได้ในส่วนของ dense layers นี้จะเป็นฟีเจอร์ที่เฉพาะเจาะจงกับปัญหาที่ใช้ในการสร้าง pretrained model ขึ้นมา และข้อสองฟีเจอร์ที่เรียนรู้ได้ในส่วนของ dense layers จะไม่มีข้อมูลเกี่ยวกับตำแหน่งเหลืออยู่ ดังนั้นฟีเจอร์ที่เรียนรู้ได้ใน dense layers นี้ จึงไม่มีประโยชน์กับการประมวลผลภาพที่จำเป็นต้องใช้ข้อมูลเกี่ยวกับตำแหน่งของวัตถุ

ไลบารี่ keras และ deeplearning4j ได้เตรียม pretrained convnet โมเดลไว้ให้กับผู้ใช้หลายตัว โดยสำหรับไลบารี่ deeplearning4j เราสามารถเลือกใช้ pretrained model ได้จากแพกเกจ 
```org.deeplearning4j.zoo``` (ข้อมูลเพิ่มเติมที่ https://deeplearning4j.org/model-zoo) สำหรับไลบารี่ keras pretrained model จะอยู่ในโมดูล ```keras.applications``` (ข้อมูลเพิ่มเติมที่ https://keras.io/applications)

ตัวอย่าง pretrained convnet ใน keras และ deeplearning4j
* VGG16
* VGG19
* ResNet50
* InceptionResNetV1
* InceptionResNetV2

### ตัวอย่างการนำ Pretrained CONVNET (VGG16) มาใช้สร้าง Cat/Dog Classifier

#### Load Data

In [ ]:
# load modules
import os 
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

# setup dataset paths
base_dir = 'data/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# setup the Image data generator
## augment the training dataset
## by rotation, shifting, shearing, zoom, flip operations
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size =(150, 150),
            batch_size=20,
            class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
            validation_dir,
            target_size=(150, 150),
            batch_size=20,
            class_mode='binary')

test_generator = test_datagen.flow_from_directory(
            test_dir,
            target_size=(150, 150),
            batch_size=20,
            class_mode='binary')

#### Load the Pretrained Model

In [ ]:
from keras.applications import VGG16

# load the pretrained model
conv_base = VGG16(weights='imagenet',   # used weights derived by pre-training on ImageNet 
                  include_top=False,    # do not include the dense layer
                  input_shape=(150, 150, 3))  # if not specified, 
                                              # the network will be able 
                                              # to process input of any size
        
# Freeze the network by setting its trainable attribute to False
# this is done to prevent weights in the convolutional base from 
# being updated during training.
conv_base.trainable = False

In [ ]:
# show a summary of model architecture
conv_base.summary()

#### สร้างโมเดล โดยการเพิ่ม Dense Layer บน convolutional base

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

#### เทรนโมเดลใหม่

In [ ]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
            train_generator,
            steps_per_epoch=100,
            epochs=30,
            validation_data=validation_generator,
            validation_steps=50)

#### บันทึกโมเดลไว้ใช้งานภายหลัง

In [ ]:
model.save('cats_and_dogs_small-vgg16.h5')

#### พล็อตกราฟแสดงประสิทธิภาพระหว่างการเทรน

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# Accuracy Plots
plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()

# Loss Plots
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation Loss')

plt.show()

#### ประเมินประสิทธิภาพของโมเดลที่ได้บน test data

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

### สรุป

* Convolutional neural network (convnet) เป็นโมเดลการเรียนรู้เชิงลึกที่เหมาะสำหรับการประมวลผลภาพ (computer vision)

* เราสามารถสร้าง convnet เองทั้งหมด หรือสร้างต่อยอดจาก pretrained models ต่างๆ เช่น VGG16, VGG19, ResNet

* สำหรับโปรเจ็กต์ที่ชุดข้อมูลมีขนาดเล็ก ปัญหาที่มักพบบ่อยคือ overfitting (โมเดลที่ได้มีประสิทธิภาพที่ดีบนชุดเทรน แต่ไม่ดีบนชุดทดสอบ) วิธีการแก้ปัญหา overfitting มีหลายวิธี เช่น การใช้ dropout layers, การทำ data augmentation, หรือการใช้ Pretrained models
   * ในไลบารี keras และ deeplearning4j มี Pretrained Models เตรียมไว้ให้ใช้งานได้หลายโมเดล เราสามารถนำส่วน convolutional layers ของ pretrained models มาใช้สร้าง convnet ตัวใหม่ได้ โดยการเพิ่ม dense layers เข้าไปบน convolutional base 